# This program connects to Box.com, gets images that match the construction type and downloads them.

In [ ]:
!pip install boxsdk
!pip install auth
!pip install redis
!pip install mysql.connector
!pip install requests

# Run the cell below to authenticate with Box.com so we can use the API

In [ ]:
from boxsdk import OAuth2

import json
#Set the file we want to use for authenticating a Box app
#The json file stores the client_id and client_secret so we don't have it in the code.
# The json file looks like this:
#{
#"client_id":"___the_codes_for_client_id___",
#"client_secret":"___the_codes_for_client_secret___"
#}

oauth_settings_file = '/home/student/data_5000_project/misc/box_app_test.json'
with open(oauth_settings_file, "r") as read_file:
    oauth_data = json.load(read_file)
print(oauth_data["client_id"])
print(oauth_data["client_secret"])

oauth = OAuth2(
    client_id=oauth_data["client_id"],
    client_secret=oauth_data["client_secret"]
)

auth_url, csrf_token = oauth.get_authorization_url('https://jeffblackadar.ca')
print("Click on this:")
print(auth_url)
print(csrf_token)
print("Copy the code that follows code= in the URL.  Paste it into the oauth.authenticate('___the_code___') below.  Be quick, the code lasts only a few seconds.")



In [ ]:
from boxsdk import Client

# Make sure that the csrf token you get from the `state` parameter
# in the final redirect URI is the same token you get from the
# get_authorization_url method to protect against CSRF vulnerabilities.
#assert 'THE_CSRF_TOKEN_YOU_GOT' == csrf_token
access_token, refresh_token = oauth.authenticate('u2ij4YXcepP4yej1bHDIJ50AYQYFgEbH')
client = Client(oauth)

# Test the connection to Box.com - list all folders and files using recursion

In [ ]:
def process_subfolder_test(client, folder_id, folder_name):
    print("this folder: "+folder_name)
    items = client.folder(folder_id=folder_id).get_items()
    for item in items:
        print('{0} {1} is named "{2}"'.format(item.type.capitalize(), item.id, item.name))
        if(item.type.capitalize()=="Folder"):
            process_subfolder_test(client, item.id,folder_name+"/"+item.name)
        if(item.type.capitalize()=="File"):
            #print(item)
            print('File: {0} is named: "{1}" path: {2} '.format(item.id, item.name, folder_name+"/"+item.name))            
    return

process_subfolder_test(client, '0',"")

In [ ]:
#We need the files from this folder
#68695443501
#Get the list of files by name put them in a dictionary with their id number so we can download it

ferebee_images = {}

items = client.folder(68695443501).get_items()
for item in items:
    #print('{0} {1} is named "{2}"'.format(item.type.capitalize(), item.id, item.name))
    if(item.type.capitalize()=="File"):
        print('File: {0} is named: "{1}"'.format(item.id, item.name))            
        ferebee_images[item.name] = item.id


In [ ]:
import csv
import os

# Contruction types
construction_types = {
"opus incertum":"opus_incertum",
"opus latericium / opus testaceum":"opus_latericium_-_opus_testaceum",
"opus vittatum mixtum":"opus_vittatum_mixtum",
"opus reticulatum":"opus_reticulatum",
"opus quadratum":"opus_quadratum",
"a telaio (framework)":"a_telaio_(framework)",
"opus vittatum":"opus_vittatum",
"opus quasi-reticulatum":"opus_quasi-reticulatum",
"opus africanum":"opus_africanum",
"opus spicatum":"opus_spicatum",
"opus craticium":"opus_craticium",
"No visible ConType":"no_visible_contype"
}


print (construction_types["opus latericium / opus testaceum"])
img_local_parent_folder ="/home/student/data_5000_project/data/images/construction_types/raw/"

with open('architecture_cinzia_presti.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            con_type = construction_types[row[1]]      
            if(con_type!="no_visible_contype"):
                print(ferebee_images[row[0]])
                id_box_file = ferebee_images[row[0]]  
                img_local_folder=img_local_parent_folder+con_type
                img_file_name = row[0]
                img_local_path = img_local_folder+"/"+row[0]
                print(img_local_path)
                #see if the image is there already
                if not os.path.exists(img_local_path):
                    if not os.path.exists(img_local_folder):
                        path = Path(img_local_folder)
                        path.mkdir(parents=True,exist_ok=True)
                    imagecontent = client.file(id_box_file).content()
                    with open(img_local_path,'wb') as f:
                        f.write(imagecontent)
                    del imagecontent
                    f.close()
                    print('wrote it')
            
                  
            #print(f'\t{row[0]} works in the {row[1]} department, and was born in {row[2]}.')
            line_count += 1
    print(f'Processed {line_count} lines.')

In [ ]:
print(construction_types)
print (construction_types["opus latericium / opus testaceum"])